In [1]:
import json
import pandas as pd
import numpy as np
from copy import deepcopy

from modules.team import Team, BenchTeam
from modules.player import Player, Position
from modules.transfer import Transfer
from modules.fixture_difficulty_matrix import FixtureDifficultyMatrix
from IPython.display import display

import config

In [2]:
CURRENT_DATE = config.CURRENT_DATE
teams_filename = f"./results/{CURRENT_DATE}/results_{CURRENT_DATE}.json"
SELECTED_MODEL_INDEX = 0
TOTAL_BUDGET = 999

In [3]:
current_team_names = {"André Onana",
                      "Matz Sels",
                      "Ashley Young",
                      "Michael Keane",
                      "Wout Faes",
                      "Jack Stephens",
                      "Ola Aina",
                      "Bukayo Saka",
                      "Bryan Mbeumo",
                      "James Maddison",
                      "Alex Iwobi",
                      "Mohamed Salah",
                      "Yoane Wissa",
                      "Matheus Santos Carneiro Da Cunha",
                      "Chris Wood"
                      }

In [4]:
with open("data/data.csv","r",encoding="utf-8") as f:
    all_player_data = pd.read_csv(f)

In [5]:
def calcPScores(pSeries: pd.Series) -> pd.Series:
    stdDev = np.std(pSeries)
    avg = pSeries.mean()
    return (pSeries - avg) / stdDev

ictIndexPScores = calcPScores(all_player_data["ict_index"])
avgPointsPScores = calcPScores(all_player_data["points_per_game"])
totalPointsPScores = calcPScores(all_player_data["total_points"])

combinedPScores = ictIndexPScores + avgPointsPScores + totalPointsPScores
all_player_data["combined"] = combinedPScores

In [6]:
print(all_player_data)

      id                          name  cost  ict_index  total_points  \
0      0         Fábio Ferreira Vieira    54        0.0             0   
1      1     Gabriel Fernando de Jesus    68       17.1             9   
2      2  Gabriel dos Santos Magalhães    62       59.7            60   
3      3                   Kai Havertz    79       87.8            59   
4      4                     Karl Hein    40        0.0             0   
..   ...                           ...   ...        ...           ...   
688  688              Bastien Meupiyou    40        0.0             0   
689  689  André Trindade da Costa Neto    50       19.5            16   
690  690   Carlos Roberto Forbs Borges    55        6.6             5   
691  691                    Alfie Pond    39        0.0             1   
692  692                    Tom Edozie    45        0.0             0   

     points_per_game  form status  starts_per_90 position team  combined  
0                0.0   0.0      u           0.00

In [7]:
current_team = Team.fromNameSet(current_team_names, all_player_data)

In [8]:
current_team_cost = current_team.getTotalCost()
current_team_cost

962

In [9]:
with open(teams_filename,"r",encoding="utf-8") as f:
    all_data = json.load(f)
data = all_data["data"][SELECTED_MODEL_INDEX]
selected_team_df = pd.DataFrame(data=data)

In [10]:
selectedTeamNames = selected_team_df["name"].values
selected_team = Team.fromNameSet(selectedTeamNames, all_player_data)

In [11]:
display(selected_team)

In [12]:
new_players = selected_team - current_team
print(new_players)


Total Score: 0

Goalkeepers:
- Jordan Pickford (Captain) 	Score: 0	Cost: 49	Fixture Difficulty: 0.0
- Dean Henderson (Vice Captain) 	Score: 0	Cost: 44	Fixture Difficulty: 0.0
Defenders:
- Trevoh Chalobah	Score: 0	Cost: 44	Fixture Difficulty: 0.0
- Vitalii Mykolenko	Score: 0	Cost: 43	Fixture Difficulty: 0.0
- Daniel Muñoz	Score: 0	Cost: 47	Fixture Difficulty: 0.0
- Emerson Palmieri dos Santos	Score: 0	Cost: 44	Fixture Difficulty: 0.0
Attackers:
- Ollie Watkins	Score: 0	Cost: 90	Fixture Difficulty: 0.0
- Jørgen Strand Larsen	Score: 0	Cost: 57	Fixture Difficulty: 0.0
Midfielders:
- Cole Palmer	Score: 0	Cost: 111	Fixture Difficulty: 0.0
- Amad Diallo	Score: 0	Cost: 51	Fixture Difficulty: 0.0


In [13]:
new_team_cost = selected_team.getTotalCost()
new_team_cost

added_cost = new_team_cost - current_team_cost
print("Added cost:",added_cost)

surplus = current_team_cost - new_team_cost


Added cost: 31


In [14]:
HEURISTIC = "combined"

In [15]:
all_player_data["score"] = all_player_data[HEURISTIC] * all_player_data["form"]
deviation = np.std(all_player_data["score"])
scale_factor = deviation
scale_factor, deviation

(14.723738573517647, 14.723738573517647)

In [16]:
START_SAMPLE_GAMEWEEK = 17
END_SAMPLE_GAMEWEEK = 20

In [17]:
matrix = FixtureDifficultyMatrix(scale_factor, START_SAMPLE_GAMEWEEK, END_SAMPLE_GAMEWEEK)

new_players.recalculateFixtureDifficulty(matrix)
new_players.calculateScore(HEURISTIC)

selected_team.recalculateFixtureDifficulty(matrix)
selected_team.calculateScore(HEURISTIC)

current_team.recalculateFixtureDifficulty(matrix)
current_team.calculateScore(HEURISTIC)

In [18]:
print("Current Team:")
display(current_team)
print()
print("Selected Team:")
display(selected_team)

Current Team:



Selected Team:


In [19]:
score_dif = selected_team.getTotalScore() - current_team.getTotalScore()
print("Score difference:",score_dif)

Score difference: 33.44196924240907


In [20]:
def getBest(pCurrentTeam: Team, pNewTeam: Team, pNewPlayers: Team, pPosition: Position):
    currentPositionData = pCurrentTeam.getPlayersListByPosition(pPosition)
    newPlayersPositionData = pNewPlayers.getPlayersListByPosition(pPosition)
    oldTotalCost = pCurrentTeam.getTotalCost()
    maxCost = max(oldTotalCost, TOTAL_BUDGET)

    allTransfers = []

    for i in range(len(currentPositionData)):
        oldPlayer = currentPositionData[i]
        oldPlayerCost = oldPlayer.getCost()
        for j in range(len(newPlayersPositionData)):

            newPlayer = newPlayersPositionData[j]
            newCost = oldTotalCost - oldPlayerCost + newPlayer.getCost()

            if (newCost <= maxCost):
                allTransfers.append(Transfer(oldPlayer, newPlayer))

    bestTransfer = max(allTransfers)
    if(bestTransfer.getScoreDif() > 0):
        return bestTransfer
    else:
        return None

In [21]:
def get_updated_team(team: pd.DataFrame, old_player: pd.Series, new_player: pd.Series):
    team: pd.DataFrame = team.drop(index=old_player.name)
    team.loc[len(team)] = new_player
    return team

In [22]:
def get_bench(team: pd.DataFrame):
    positions = ["FWD","DEF","MID","GKP"]
    team = team.reset_index()
    bench = pd.DataFrame(columns=team.columns)
    for position in positions:
        worst_player_index = team.loc[team["position"]==position]["score"].idxmin()
        worst_player = team.loc[worst_player_index].copy()
        bench.loc[len(bench)] = worst_player
        team = team.drop(index=worst_player_index)
    return team, bench

In [23]:
def getBestTransferNew(pCurrentTeam: Team, pNewTeam: Team, pNewPlayers: Team) -> Transfer | None:
    positions = Position.listValues()
    bestTransfers = []
    for position in positions:
        transfer = getBest(pCurrentTeam, pNewTeam, pNewPlayers, position)
        if(transfer is not None):
            bestTransfers.append(transfer)
    actualBestTransfer = max(bestTransfers)
    return actualBestTransfer

In [24]:
def getNewTeam(pCurrentTeam: Team, pSelectedTeam: Team, pNewPlayers: Team):
    transferData = getBestTransferNew(pCurrentTeam, pSelectedTeam, pNewPlayers)
    print("Best transfer:")
    print(transferData)
    oldPlayer = transferData.getOldPlayer()
    newPlayer = transferData.getNewPlayer()
    if transferData is None:
        return
    
    changingPosition: Position = transferData.getPosition()
    
    newTeam = deepcopy(pCurrentTeam)
    playersOfPosition = newTeam.getPlayersListByPosition(changingPosition)

    for i in range(len(playersOfPosition)):
        player = playersOfPosition[i]
        if (player.getId() == oldPlayer.getId()):
            newTeam.removePlayerByIndex(i, changingPosition)
            newTeam.addPlayer(newPlayer)
    return newTeam

In [25]:
newTeam = getNewTeam(current_team, selected_team, new_players)
newTeam = newTeam.toBenchTeam()
display(newTeam)

Best transfer:
Transfer from Bryan Mbeumo -> Cole Palmer:
Old player: Bryan Mbeumo	Score: 48.74	Cost: 76	Fixture Difficulty: 2.0613234002924727
New player: Cole Palmer	Score: 159.6	Cost: 111	Fixture Difficulty: -7.803581443964354
- Cost change: 35
- Score change: 110.86
Cole Palmer (Vice Captain) 	Score: 159.6	Cost: 111	Fixture Difficulty: -7.803581443964354
